In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob

In [2]:
import os
filename = []
list = os.listdir('/Users/HP/Documents/Cap/capstone/c_prepared_data')
for f in list:
    if "all" in f:
        filename.append(f)

In [3]:
filename

['1754_all.csv',
 '1804_all.csv',
 '181129_all.csv',
 '1828_all.csv',
 '1834_all.csv',
 '185429_all.csv']

In [4]:

df = pd.DataFrame()
for f in filename:
    df_c = pd.read_csv("../c_prepared_data/"+f)
    df=df.append(df_c)

    

In [5]:
features = df[['mxamp', 'std', 'skew', 'kurtosis', 'hr',  'patpk', 'patft']]
target = df[['sbp', 'dbp']]

In [6]:
features

,mxamp,std,skew,kurtosis,hr,patpk,patft
0,1192.900858,323.576299,-0.184310,-1.149187,65.040650,2.1700,0.206667
1,1192.900858,339.432634,-0.187815,-1.239822,62.176166,2.2600,0.206667
2,1192.900858,324.194376,-0.306513,-1.101659,69.364162,1.2100,0.217500
3,1203.866910,325.383586,-0.321261,-1.113743,69.364162,1.1950,0.215000
4,1223.268210,348.023151,-0.213124,-1.267128,70.796460,1.2000,0.210000
...,...,...,...,...,...,...,...
82,280.924809,76.974487,0.060091,-1.210847,83.044983,1.1950,0.190000
83,303.249129,85.742040,0.236107,-1.190659,84.705882,1.7250,0.185000
84,385.590239,100.699059,0.080688,-1.141817,81.911263,1.1425,0.173333
85,392.598289,110.293092,0.112247,-1.266855,75.709779,2.0000,0.175000


In [7]:
def cindex(true_labels, predictions):
    n,h = 0,0
    
    for i in range(1,len(true_labels)):
        t = true_labels[i]
        p = predictions[i]
        for j in range(i+1,len(true_labels)):
            nt = true_labels[j]
            npr = predictions[j]
            if t != nt:
                n += 1
                if (p < npr and t < nt) or (p >npr and t > nt):
                    h += 1
                else :
                    h += 0.5
                    
    return h/n

In [17]:
from sklearn.model_selection import LeaveOneOut
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
loo = LeaveOneOut()
X = features.values
y = target.values
sbp_er = []
dbp_er = []
true_label = []
pred_label = []
acc_sbp = 0
acc_dbp = 0
for i, (train_idx, val_idx) in enumerate(loo.split(X)):
    X_train, X_test = X[train_idx], X[val_idx]
    y_train, y_test = y[train_idx], y[val_idx]
    
    model = make_pipeline(StandardScaler(),LinearRegression())
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    mae_sbp = mean_absolute_error(y_pred[:,0], y_test[:,0])
    mae_dbp = mean_absolute_error(y_pred[:,1], y_test[:,1])

    if mae_sbp<= 10:
        acc_sbp = acc_sbp+1
    if mae_dbp <=10:
        acc_dbp = acc_dbp+1
        
    sbp_er.append(mae_sbp)
    dbp_er.append(mae_dbp)
    
    true_label.append(y_test)
    pred_label.append(y_pred)
true = [item for sublist in true_label for item in sublist]
pred = [item for sublist in pred_label for item in sublist]

true = np.array(true)
pred = np.array(pred)

cidx_sbp = cindex(true[:,0], pred[:,0])
cidx_dbp = cindex(true[:,1], pred[:,1])
    
print(f"Average MAE SBP: {np.mean(sbp_er)} MAE DBP: {np.mean(dbp_er)}")
print(f"C index SBP: {cidx_sbp}  C index DBP: {cidx_dbp}")
print(f"Probability SBP: {acc_sbp/len(pred)}  Probability DBP: {acc_dbp/len(pred)}" )

Average MAE SBP: 7.314507921397743 MAE DBP: 4.650300859491027
C index SBP: 0.8664403833801433  C index DBP: 0.8572634071387188
Probability SBP: 0.6925  Probability DBP: 0.89


In [18]:
df_c_lr = pd.DataFrame({
    "true_sbp": true[:,0], 
    "pred_sbp": pred[:,0],
    "true_dbp": true[:,1],
    "pred_dbp": pred[:,1]
                       })
df_c_lr

,true_sbp,pred_sbp,true_dbp,pred_dbp
0,98.196667,109.260082,65.373333,78.949205
1,104.613333,109.047731,67.176667,78.462032
2,109.693333,104.235681,72.980000,76.283629
3,115.752500,103.577468,78.895000,75.936266
4,116.102500,104.539453,82.622500,76.616914
...,...,...,...,...
395,100.937500,109.723702,64.300000,78.390900
396,102.285000,114.638479,65.967500,81.510448
397,105.820000,111.074199,69.965000,79.440857
398,109.287500,113.116239,71.192500,80.021351


In [19]:
df_c_lr.to_csv("../Results/c_lr.csv", index = False)

In [14]:
from sklearn.ensemble import RandomForestRegressor

loo = LeaveOneOut()
X = features.values
y = target.values
sbp_er = []
dbp_er = []
true_label = []
pred_label = []
acc_sbp = 0
acc_dbp = 0
for i, (train_idx, val_idx) in enumerate(loo.split(X)):
    X_train, X_test = X[train_idx], X[val_idx]
    y_train, y_test = y[train_idx], y[val_idx]
    
    forest = make_pipeline(StandardScaler(),RandomForestRegressor(n_estimators = 200))
    forest.fit(X_train, y_train)
    y_pred = forest.predict(X_test)
    
    mae_sbp = mean_absolute_error(y_pred[:,0], y_test[:,0])
    mae_dbp = mean_absolute_error(y_pred[:,1], y_test[:,1])

    if mae_sbp<= 10:
        acc_sbp = acc_sbp+1
    if mae_dbp <=10:
        acc_dbp = acc_dbp+1
        
    sbp_er.append(mae_sbp)
    dbp_er.append(mae_dbp)
    
    true_label.append(y_test)
    pred_label.append(y_pred)
true = [item for sublist in true_label for item in sublist]
pred = [item for sublist in pred_label for item in sublist]

true = np.array(true)
pred = np.array(pred)

cidx_sbp = cindex(true[:,0], pred[:,0])
cidx_dbp = cindex(true[:,1], pred[:,1])
    
print(f"Average MAE SBP: {np.mean(sbp_er)} MAE DBP: {np.mean(dbp_er)}")
print(f"C index SBP: {cidx_sbp}  C index DBP: {cidx_dbp}")
print(f"Probability SBP: {acc_sbp/len(pred)}  Probability DBP: {acc_dbp/len(pred)}" )

Average MAE SBP: 4.310507262499997 MAE DBP: 3.3979093000000025
C index SBP: 0.9107923273592866  C index DBP: 0.8993866344240409
Probability SBP: 0.9075  Probability DBP: 0.9525


In [15]:
df_c_rf = pd.DataFrame({
    "true_sbp": true[:,0], 
    "pred_sbp": pred[:,0],
    "true_dbp": true[:,1],
    "pred_dbp": pred[:,1]
                       })
df_c_rf

,true_sbp,pred_sbp,true_dbp,pred_dbp
0,98.196667,105.778242,65.373333,72.011379
1,104.613333,104.311363,67.176667,71.998629
2,109.693333,105.184154,72.980000,73.932664
3,115.752500,107.179673,78.895000,74.764640
4,116.102500,108.909991,82.622500,77.122442
...,...,...,...,...
395,100.937500,113.659237,64.300000,75.172083
396,102.285000,113.827854,65.967500,78.029727
397,105.820000,109.058062,69.965000,73.009969
398,109.287500,110.983712,71.192500,73.782028


In [16]:
df_c_rf.to_csv("../Results/c_rf.csv", index = False)

In [11]:
from sklearn.svm import SVR
svr = make_pipeline(StandardScaler(),SVR(kernel = 'rbf', C=10, epsilon = 0.01))
X = features.values
y = target.values
sbp_er = []
dbp_er = []
true_label_sbp = []
pred_label_sbp = []

true_label_dbp = []
pred_label_dbp= []

acc_sbp = 0
acc_dbp = 0

for i, (train_idx, val_idx) in enumerate(loo.split(X)):
    X_train, X_test = X[train_idx], X[val_idx]
    y_train, y_test = y[train_idx], y[val_idx]
    

    svr.fit(X_train, y_train[:,0])
    y_pred_sbp = svr.predict(X_test)
    true_label_sbp.append(y_test[:,0])
    pred_label_sbp.append(y_pred_sbp)
    
    svr.fit(X_train, y_train[:,1])
    y_pred_dbp = svr.predict(X_test)
    true_label_dbp.append(y_test[:,1])
    pred_label_dbp.append(y_pred_dbp)
    
    mae_sbp = mean_absolute_error(y_pred_sbp, y_test[:,0])
    mae_dbp = mean_absolute_error(y_pred_dbp, y_test[:,1])
    
    sbp_er.append(mae_sbp)
    dbp_er.append(mae_dbp)
    
    if mae_sbp<= 10:
        acc_sbp = acc_sbp+1
    if mae_dbp <=10:
        acc_dbp = acc_dbp+1    
    
true_sbp = [item for sublist in true_label_sbp for item in sublist]
pred_sbp = [item for sublist in pred_label_sbp for item in sublist]

true_dbp = [item for sublist in true_label_dbp for item in sublist]
pred_dbp = [item for sublist in pred_label_dbp for item in sublist]

true_sbp = np.array(true_sbp)
pred_sbp = np.array(pred_sbp)

true_dbp = np.array(true_dbp)
pred_dbp = np.array(pred_dbp)

cidx_sbp = cindex(true_sbp, pred_sbp)
cidx_dbp = cindex(true_dbp, pred_dbp)
    
print(f"Average MAE SBP: {np.mean(sbp_er)} MAE DBP: {np.mean(dbp_er)}")
print(f"C index SBP: {cidx_sbp}  C index DBP: {cidx_dbp}")
print(f"Probability SBP: {acc_sbp/len(pred)}  Probability DBP: {acc_dbp/len(pred)}" )

Average MAE SBP: 5.3237796857863575 MAE DBP: 4.064124508336265
C index SBP: 0.885584201312359  C index DBP: 0.8794113201843875
Probability SBP: 0.85  Probability DBP: 0.925


In [12]:
df_c_svm = pd.DataFrame({
    "true_sbp": true_sbp, 
    "pred_sbp": pred_sbp,
    "true_dbp": true_dbp,
    "pred_dbp": pred_dbp
                       })
df_c_svm

,true_sbp,pred_sbp,true_dbp,pred_dbp
0,98.196667,105.881221,65.373333,75.518612
1,104.613333,106.072781,67.176667,75.175183
2,109.693333,105.407518,72.980000,75.631514
3,115.752500,105.289131,78.895000,74.603515
4,116.102500,106.435934,82.622500,77.121363
...,...,...,...,...
395,100.937500,111.337542,64.300000,78.666039
396,102.285000,115.526797,65.967500,80.389833
397,105.820000,110.936632,69.965000,78.285769
398,109.287500,116.171017,71.192500,80.809283


In [13]:
df_c_svm.to_csv("../Results/c_svm.csv", index = False)